In [1]:
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm

In [2]:
import gensim.downloader as api

In [3]:
wv = api.load('word2vec-google-news-300')

In [4]:
cities = pd.read_csv('./csv/worldcitiespop.csv',header=0,dtype={'AccentCity':'str','Region':'object'}).dropna().drop('Region',axis=1)
cities = cities[cities.Population>100000].reset_index(drop=True)
cities

,Country,City,AccentCity,Population,Latitude,Longitude
0,ae,abu dhabi,Abu Dhabi,603687.0,24.466667,54.366667
1,ae,dubai,Dubai,1137376.0,25.258172,55.304717
2,ae,sharjah,Sharjah,543942.0,25.357310,55.403304
3,af,baglan,Baglan,108481.0,36.130684,68.708286
4,af,gardez,Gardez,103732.0,33.597439,69.225922
...,...,...,...,...,...,...
3522,zw,gweru,Gweru,201879.0,-19.450000,29.816667
3523,zw,harare,Harare,2213701.0,-17.817778,31.044722
3524,zw,kadoma,Kadoma,100276.0,-18.350000,29.916667
3525,zw,kwekwe,Kwekwe,116332.0,-18.916667,29.816667


In [5]:
countries = pd.read_csv('./csv/countries.csv',keep_default_na=False)
countries

,Name,Code,Latitude,Longitude
0,Andorra,AD,42.5000,1.5000
1,United Arab Emirates,AE,24.0000,54.0000
2,Afghanistan,AF,33.0000,65.0000
3,Antigua and Barbuda,AG,17.0500,-61.8000
4,Anguilla,AI,18.2500,-63.1667
...,...,...,...,...
244,Yemen,YE,15.0000,48.0000
245,Mayotte,YT,-12.8333,45.1667
246,South Africa,ZA,-29.0000,24.0000
247,Zambia,ZM,-15.0000,30.0000


In [8]:
cities_embdgs = np.empty( (len(cities),300) )
cities_embdgs[:] = np.NaN #fill with NaN values
missed = []
for i,city in tqdm(enumerate(cities.City),total = len(cities),file=sys.stdout) :
    city_split = city.split()
    for j in range(len(city_split)):
        city_split[j] = city_split[j][0].upper() + city_split[j][1:]
    city_name = "_".join(city_split)
    try:
        cities_embdgs[i] = wv[city_name]
    except KeyError:
        try :
            cities_embdgs[i] = wv[city_name.upper()]
        except KeyError:
            try :
                cities_embdgs[i] = wv["_".join(city_name.split('-'))]
            except KeyError :
                try :
                    cities_embdgs[i] = wv["_".join(city_name.split('-')).upper()]
                except KeyError :                
                    missed.append( (i,city) )

100%|██████████| 3527/3527 [00:00<00:00, 8207.20it/s]


In [508]:
np.save(arr=cities_embdgs,file='./embd_files/w2v_cities.npy')

In [502]:
for word in wv.index_to_key:
    if 'syzr' in word.lower():
        print(word)

In [498]:
ind = 33
missed[ind]

(2867, 'staryy oskol')

In [499]:
i = missed[ind][0]
cities_embdgs[i] = wv['Stary_Oskol']
print(missed.pop(ind),'\n now :',missed[ind])

(2867, 'staryy oskol') 
 now : (2869, 'sterlitamak')


In [509]:
country_names = [ name.split(',')[0].split('(')[0].strip() for name in countries.Name]

In [510]:
for i in range(len(country_names)-1):
    if country_names[i]==country_names[i+1]:
        print(country_names[i],i)

Korea 120
Virgin Islands 238


In [511]:
country_names[120] = 'North '+country_names[120]
country_names[121] = 'South '+country_names[121]

country_names[238] = 'British '+country_names[238]
country_names[239] = 'U.S. '+country_names[239]

In [512]:
countries_embdgs = np.empty( (len(countries),300) )
countries_embdgs[:] = np.NaN #fill with NaN values
missed = []
for i,country in tqdm(enumerate(country_names),total = len(countries),file=sys.stdout) :
    country_split = country.split()
    for j in range(len(country_split)):
        country_split[j] = country_split[j][0].upper() + country_split[j][1:]
    country_name = "_".join(country_split)
    try:
        countries_embdgs[i] = wv[country_name]
    except KeyError:
        try :
            countries_embdgs[i] = wv[country_name.upper()]
        except KeyError:
            try :
                countries_embdgs[i] = wv["_".join(country_name.split('-'))]
            except KeyError :
                try :
                    countries_embdgs[i] = wv["_".join(country_name.split('-')).upper()]
                except KeyError :                
                    missed.append( (i,country) )

100%|██████████| 249/249 [00:00<00:00, 5559.04it/s]


In [513]:
missed

[(3, 'Antigua and Barbuda'),
 (43, "Côte d'Ivoire"),
 (53, 'Christmas Island'),
 (89, 'South Georgia and the South Sandwich Islands'),
 (95, 'Heard Island and McDonald Islands'),
 (105, 'British Indian Ocean Territory'),
 (119, 'Saint Kitts and Nevis'),
 (125, "Lao People's Democratic Republic"),
 (179, 'Saint Pierre and Miquelon'),
 (198, 'Saint Helena'),
 (200, 'Svalbard and Jan Mayen'),
 (211, 'Syrian Arab Republic'),
 (215, 'French Southern Territories'),
 (231, 'United States Minor Outlying Islands'),
 (236, 'Saint Vincent and the Grenadines'),
 (239, 'U.S. Virgin Islands'),
 (242, 'Wallis and Futuna')]

In [544]:
ind = 13
missed[ind]

(242, 'Wallis and Futuna')

In [545]:
i = missed[ind][0]
countries_embdgs[i] = wv["Wallis_Futuna"]
print(missed.pop(ind),'\n now :',missed[ind])

IndexError: list index out of range

In [546]:
np.save(arr=countries_embdgs,file='./embd_files/w2v_countries.npy')

In [548]:
len(np.load('./embd_files/w2v_countries.npy'))

249